Perform all import statements, and define helper functions for querying the API

In [1]:
import pandas as pd
import requests
import json

# Set base url
base_url = "https://api.census.gov/data/2018/{data_source}?get="

# Get api-key and format url
with open('keyfile.txt') as file:
    my_key = file.read().strip()

def get_query_url(source, *cols, geography='us', geography_choice='*', **conditions):
    
    colstring = ""
    for col in cols:
        colstring += col+","
    colstring = colstring[:-1]
    
    geostring = f"&for={geography}:{geography_choice}"
    
    condstring = ""
    for key, value in conditions.items():
        condstring += f"&{key}={value}"
    
    keystring = f"&key={my_key}"
    
    query_url = base_url.format(data_source=source)+colstring+geostring+condstring+keystring
    return query_url


Query the ABS Characteristics of Business Owners table

In [2]:
owner_url = get_query_url('abscbo', 'NAICS2017', 'NAICS2017_LABEL', 'QDESC', 'QDESC_LABEL','OWNER_RACE', 'OWNER_RACE_LABEL', 'OWNER_SEX', 'OWNER_SEX_LABEL', 'OWNCHAR', 'OWNCHAR_LABEL', 'OWNPDEMP', 'OWNPDEMP_PCT')

owner_query = requests.get(owner_url)
print(owner_query)

<Response [200]>


Load the data from the response into a Pandas dataframe, begin transformation process

In [3]:
owners_df = pd.DataFrame.from_dict(owner_query.json())

owners_df.columns = owners_df.iloc[0]
owners_df.drop(index=0, columns = 'us', inplace=True)

owners_df.head()

,NAICS2017,NAICS2017_LABEL,QDESC,QDESC_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNCHAR,OWNCHAR_LABEL,OWNPDEMP,OWNPDEMP_PCT
1,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CI03,2013 to 2017,21939,29.9
2,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CM03,2018,912,1.2
3,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CN,Don't know,7441,10.1
4,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CO,Total reporting,73427,100.0
5,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CP,Item not reported,3106,0.0


Convert the types for 'OWNPDEMP' and 'OWNPDEMP_PCT' to numeric

In [4]:
owners_df[['OWNPDEMP', 'OWNPDEMP_PCT']] = owners_df[['OWNPDEMP', 'OWNPDEMP_PCT']].apply(pd.to_numeric)
owners_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46098 entries, 1 to 46098
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NAICS2017         46098 non-null  object 
 1   NAICS2017_LABEL   46098 non-null  object 
 2   QDESC             46098 non-null  object 
 3   QDESC_LABEL       46098 non-null  object 
 4   OWNER_RACE        46098 non-null  object 
 5   OWNER_RACE_LABEL  46098 non-null  object 
 6   OWNER_SEX         46098 non-null  object 
 7   OWNER_SEX_LABEL   46098 non-null  object 
 8   OWNCHAR           46098 non-null  object 
 9   OWNCHAR_LABEL     46098 non-null  object 
 10  OWNPDEMP          46098 non-null  int64  
 11  OWNPDEMP_PCT      46098 non-null  float64
dtypes: float64(1), int64(1), object(10)
memory usage: 4.2+ MB


Query the Census API for the ABS Characteristics of Businesses Table, only bringing in relevant fields

In [5]:
business_url = get_query_url('abscb', 'NAICS2017', 'NAICS2017_LABEL', 'RACE_GROUP', 'RACE_GROUP_LABEL', 'SEX', 'SEX_LABEL', 'QDESC', 'QDESC_LABEL', 'BUSCHAR', 'BUSCHAR_LABEL', 'FIRMPDEMP', 'FIRMPDEMP_PCT', 'EMP', 'EMP_PCT', 'PAYANN', 'PAYANN_PCT', 'RCPPDEMP', 'RCPPDEMP_PCT')

business_query = requests.get(business_url)
print(business_query)

<Response [200]>


Load the data from the response into a Pandas dataframe, begin transformation process

In [6]:
businesses_df = pd.DataFrame.from_dict(business_query.json())

businesses_df.columns = businesses_df.iloc[0]
businesses_df.drop(index=0, columns='us', inplace=True)

businesses_df.head()

,NAICS2017,NAICS2017_LABEL,RACE_GROUP,RACE_GROUP_LABEL,SEX,SEX_LABEL,QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,FIRMPDEMP,FIRMPDEMP_PCT,EMP,EMP_PCT,PAYANN,PAYANN_PCT,RCPPDEMP,RCPPDEMP_PCT
1,00,Total for all sectors,00,Total,001,Total,B01,OWNRNUM,A1,All firms,5722142,0.0,128196406,0.0,6937240308,0.0,38405539368,0.0
2,00,Total for all sectors,00,Total,001,Total,B01,OWNRNUM,BQ,1 person,1628501,59.1,10991334,20.1,454218390,14.2,2270021185,12.2
3,00,Total for all sectors,00,Total,001,Total,B01,OWNRNUM,BQ01,2 to 4 people,1024784,37.2,11240900,20.6,478308387,15.0,2462681905,13.2
4,00,Total for all sectors,00,Total,001,Total,B01,OWNRNUM,BQ02,5 to 10 people,49436,1.8,1423837,2.6,75150602,2.4,413777541,2.2
5,00,Total for all sectors,00,Total,001,Total,B01,OWNRNUM,BQ03,11 or more people,31846,1.2,13448854,24.6,986906070,30.9,6023549720,32.3


Convert 'FIRMPDEMP', 'FIRMPDEMP_PCT', 'EMP', 'EMP_PCT', 'PAYANN', 'PAYANN_PCT', 'RCPPDEMP', 'RCPPDEMP_PCT' to numeric

In [7]:
businesses_df[['FIRMPDEMP', 'FIRMPDEMP_PCT', 'EMP', 'EMP_PCT', 'PAYANN', 'PAYANN_PCT', 'RCPPDEMP', 'RCPPDEMP_PCT']] = businesses_df[['FIRMPDEMP', 'FIRMPDEMP_PCT', 'EMP', 'EMP_PCT', 'PAYANN', 'PAYANN_PCT', 'RCPPDEMP', 'RCPPDEMP_PCT']].apply(pd.to_numeric)
businesses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17202 entries, 1 to 17202
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NAICS2017         17202 non-null  object 
 1   NAICS2017_LABEL   17202 non-null  object 
 2   RACE_GROUP        17202 non-null  object 
 3   RACE_GROUP_LABEL  17202 non-null  object 
 4   SEX               17202 non-null  object 
 5   SEX_LABEL         17202 non-null  object 
 6   QDESC             17202 non-null  object 
 7   QDESC_LABEL       17202 non-null  object 
 8   BUSCHAR           17202 non-null  object 
 9   BUSCHAR_LABEL     17202 non-null  object 
 10  FIRMPDEMP         17202 non-null  int64  
 11  FIRMPDEMP_PCT     17202 non-null  float64
 12  EMP               17202 non-null  int64  
 13  EMP_PCT           17202 non-null  float64
 14  PAYANN            17202 non-null  int64  
 15  PAYANN_PCT        17202 non-null  float64
 16  RCPPDEMP          17202 non-null  int64 

Merge above two dataframes on NAICS2017, owner race, and owner sex

In [8]:
owner_and_firm_data = owners_df.merge(businesses_df, how='inner', left_on=['NAICS2017', 'OWNER_RACE', 'OWNER_SEX'], right_on=['NAICS2017', 'RACE_GROUP', 'SEX'])

In [9]:
print(owner_and_firm_data.shape)
owner_and_firm_data.head(10)

(1222005, 29)


,NAICS2017,NAICS2017_LABEL_x,QDESC_x,QDESC_LABEL_x,OWNER_RACE,OWNER_RACE_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNCHAR,OWNCHAR_LABEL,...,BUSCHAR,BUSCHAR_LABEL,FIRMPDEMP,FIRMPDEMP_PCT,EMP,EMP_PCT,PAYANN,PAYANN_PCT,RCPPDEMP,RCPPDEMP_PCT
0,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CI03,2013 to 2017,...,A1,All firms,124551,0.0,1188819,0.0,36939229,0.0,128012399,0.0
1,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CI03,2013 to 2017,...,BQ,1 person,38676,73.9,234097,65.3,6974978,66.7,24043278,67.2
2,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CI03,2013 to 2017,...,BQ01,2 to 4 people,12919,24.7,117209,32.7,3155883,30.2,10790501,30.1
3,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CI03,2013 to 2017,...,BQ02,5 to 10 people,0,0.0,0,0.0,0,0.0,0,0.0
4,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CI03,2013 to 2017,...,BQ03,11 or more people,0,0.0,0,0.0,0,0.0,0,0.0
5,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CI03,2013 to 2017,...,BX,Unknown number of owners,252,0.5,1858,0.5,68443,0.7,190588,0.5
6,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CI03,2013 to 2017,...,BY,Total reporting,52332,100.0,358499,100.0,10462621,100.0,35790268,100.0
7,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CI03,2013 to 2017,...,BZ,Item not reported,2014,0.0,9572,0.0,224771,0.0,738434,0.0
8,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CI03,2013 to 2017,...,A1,All firms,124551,0.0,1188819,0.0,36939229,0.0,128012399,0.0
9,00,Total for all sectors,O02,YRACQBUS,40,Black or African American,001,All owners of respondent firms,CI03,2013 to 2017,...,BM,Family-owned,10111,19.8,96433,28.5,2308352,22.8,7333604,21.2


In [10]:
owner_and_firm_data.columns

Index(['NAICS2017', 'NAICS2017_LABEL_x', 'QDESC_x', 'QDESC_LABEL_x',
       'OWNER_RACE', 'OWNER_RACE_LABEL', 'OWNER_SEX', 'OWNER_SEX_LABEL',
       'OWNCHAR', 'OWNCHAR_LABEL', 'OWNPDEMP', 'OWNPDEMP_PCT',
       'NAICS2017_LABEL_y', 'RACE_GROUP', 'RACE_GROUP_LABEL', 'SEX',
       'SEX_LABEL', 'QDESC_y', 'QDESC_LABEL_y', 'BUSCHAR', 'BUSCHAR_LABEL',
       'FIRMPDEMP', 'FIRMPDEMP_PCT', 'EMP', 'EMP_PCT', 'PAYANN', 'PAYANN_PCT',
       'RCPPDEMP', 'RCPPDEMP_PCT'],
      dtype='object', name=0)

Drop redundant columns, and rename other columns to be more descriptive

In [11]:
owner_and_firm_data.drop(columns=['NAICS2017_LABEL_y', 'RACE_GROUP', 'RACE_GROUP_LABEL', 'SEX', 'SEX_LABEL'], inplace=True)
owner_and_firm_data.rename(columns={"NAICS2017_LABEL_x":"NAICS2017_LABEL", "QDESC_x":"CBO_QDESC", "QDESC_LABEL_x":"CBO_QDESC_LABEL", 'QDESC_y':'CB_QDESC', 'QDESC_LABEL_y':'CB_QDESC_LABEL'}, inplace=True)

Save pandas dataframes to CSV's

In [12]:
owners_df.to_csv('characteristics_of_business_owners.csv')
businesses_df.to_csv('business_characteristics.csv')
owner_and_firm_data.to_csv('merged_data.csv')